<a href="https://colab.research.google.com/github/gtoranzo/tor-ia/blob/main/mount_drive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Paso 1: Montar Google Drive (para acceder a credentials.json y al video)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import os
import pickle
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import Flow  # ← Usa Flow directamente, no InstalledAppFlow.from_client...
from google.auth.transport.requests import Request
from googleapiclient.http import MediaFileUpload
from urllib.parse import urlparse, parse_qs

SCOPES = ['https://www.googleapis.com/auth/youtube.upload']

def authenticate_youtube():
    creds = None
    token_path = '/content/drive/MyDrive/token.pickle'
    cred_path = '/content/drive/MyDrive/credentials.json'

    # Cargar token si existe y es válido
    if os.path.exists(token_path) and os.path.getsize(token_path) > 0:
        try:
            with open(token_path, 'rb') as token:
                creds = pickle.load(token)
        except (EOFError, pickle.UnpackingError):
            print("⚠️ Token corrupto. Eliminando...")
            os.remove(token_path)
            creds = None

    # Refrescar o autenticar
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            try:
                creds.refresh(Request())
            except Exception as e:
                print(f"⚠️ No se pudo refrescar: {e}. Reautenticando...")
                creds = None

        if not creds:
            if not os.path.exists(cred_path):
                raise FileNotFoundError(f"❌ credentials.json no encontrado: {cred_path}")

            # Flujo manual SIN run_console() ni run_local_server()
            flow = Flow.from_client_secrets_file(
                cred_path,
                scopes=SCOPES,
                redirect_uri='urn:ietf:wg:oauth:2.0:oob'  # URI oficial para consola
            )

            # Generar URL de autorización
            auth_url, _ = flow.authorization_url(
                access_type='offline',
                #include_granted_scopes='true',
                prompt='consent'  # fuerza obtener refresh_token
            )

            print("🔒 Paso 1: Abre este enlace EN TU NAVEGADOR LOCAL (Chrome/Brave):")
            print(auth_url)
            print("\n➡️ Tras autorizar, se mostrará una página con un CÓDIGO de autorización.")
            print("👉 COPIA y PEGA ESE CÓDIGO AQUÍ 👇")

            # Leer el código de autorización
            auth_code = input("\nCódigo de autorización: ").strip()

            # Obtener el token usando el código
            if not auth_code:
                raise ValueError("❌ No se introdujo ningún código de autorización.")
            try:
                flow.fetch_token(code=auth_code)
                creds = flow.credentials
            except Exception as e:
                raise ValueError(f"❌ Error al obtener el token con el código proporcionado: {e}")

        # Guardar token
        os.makedirs(os.path.dirname(token_path), exist_ok=True)
        with open(token_path, 'wb') as token:
            pickle.dump(creds, token)
        print("✅ Token guardado en Drive.")

    return build('youtube', 'v3', credentials=creds)

# Ejecutar autenticación
youtube = authenticate_youtube()
print("✅ Autenticado con YouTube")

🔒 Paso 1: Abre este enlace EN TU NAVEGADOR LOCAL (Chrome/Brave):
https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=737423817601-0npok8pt63u9uf0qja763gsef8blire0.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.upload&state=JmUjLIxjIl8e9xl8IguCJNanBKHAxK&access_type=offline&prompt=consent

➡️ Tras autorizar, se mostrará una página con un CÓDIGO de autorización.
👉 COPIA y PEGA ESE CÓDIGO AQUÍ 👇

Código de autorización: 4/1Ab32j90j19_wnJ4G2nxBAE58jfHL2w3PHu1j7k4slJ0f_jgVxgJOdUNzjXw
✅ Token guardado en Drive.
✅ Autenticado con YouTube


In [43]:
# Paso 3: Ruta del video en Google Drive
video_path = "/content/drive/MyDrive/Videos/PUBG_BATTLEGROUNDS_2024-11-18 03-56-48.mp4"  # ← AJUSTA ESTA RUTA

# Verifica que exista
if not os.path.exists(video_path):
    raise FileNotFoundError(f"Video no encontrado: {video_path}")
else:
    print(f"🎬 Encontrado: {os.path.basename(video_path)} | {os.path.getsize(video_path)//(1024*1024)} MB")

🎬 Encontrado: PUBG_BATTLEGROUNDS_2024-11-18 03-56-48.mp4 | 1779 MB


In [44]:
# Prueba rápida: listar tus playlists (requiere YouTube API v3)
# url para Activa la API: Haz clic en ENABLE (o ACTIVAR si está en español).
# https://console.developers.google.com/apis/api/youtube.googleapis.com/overview?spm=a2ty_o01.29997173.0.0.7012c921QxOZlZ&project=737423817601
# Verifica que esté activa: Debe decir "API enabled".
# https://console.cloud.google.com/apis/library/youtube.googleapis.com?spm=a2ty_o01.29997173.0.0.7012c921QxOZlZ&project=737423817601
try:
    playlists = youtube.playlists().list(part="snippet", mine=True, maxResults=1).execute()
    print("✅ YouTube API v3 está ACTIVA y funcional.")
except Exception as e:
    print("❌ Error:", str(e))

❌ Error: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/playlists?part=snippet&mine=true&maxResults=1&alt=json returned "Request had insufficient authentication scopes.". Details: "[{'message': 'Insufficient Permission', 'domain': 'global', 'reason': 'insufficientPermissions'}]">


In [45]:
# Paso 4: Subir a YouTube
request_body = {
    'snippet': {
        'title': 'Mi video desde Colab 🚀',
        'description': 'Subido automáticamente sin descargar localmente.',
        'tags': ['automatización', 'colab', 'python'],
        'categoryId': '22'  # People & Blogs ver: (https://gist.github.com/dgp/1b24bf2961521bd75d6c)
    },
    'status': {
        'privacyStatus': 'private'  # Opciones: 'private', 'unlisted', 'public'
    }
}

media = MediaFileUpload(video_path, chunksize=10*1024*1024, resumable=True)

print("📤 Subiendo video a YouTube (puede tardar varios minutos)...")
response = youtube.videos().insert(
    part="snippet,status",
    body=request_body,
    media_body=media
).execute()

print(f"✅ Subido con éxito!")
print(f"🔗 ID del video: {response['id']}")
print(f"🌐 Ver en: https://youtu.be/{response['id']}")

📤 Subiendo video a YouTube (puede tardar varios minutos)...
✅ Subido con éxito!
🔗 ID del video: Jhu0FS407Ug
🌐 Ver en: https://youtu.be/Jhu0FS407Ug
